In [13]:
import cv2
import glob
from ultralytics import YOLO
import os

In [17]:
def process_frames_folder(folder_path):
    print(f"Loading the Model...")

    model = YOLO('yolov8n.pt')

    image_patterns = os.path.join(folder_path, '*.jpeg')

    frame_files = []
    for pattern in image_patterns:
        frame_files.extend(glob.glob(pattern))

    frame_files.sort()

    if not frame_files:
        print(f"No Images in the folder {folder_path}")
        return

    print(f"Found images.. now processing in {folder_path}")

    print("\n--- Detection Results (CSV Format) ---")
    print("frame_filename,class_id,class_name,confidence,x_min,y_min,x_max,y_max")

    results = model(frame_files, stream=False, verbose=False)

    for result in results:
        frame_filename = os.path.basename(result.path)

        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.conf.cpu().numpy().astype(int)

        for i in range(len(boxes)):
           box = boxes[i]
           x_min, y_min, x_max, y_max = box
           conf = confidences[i]
           id = class_ids[i]

           class_name = model.names[id]

           if class_name in ['car', 'truck', 'bus', 'motorcycle']:
               print(
                    f"{frame_filename},"
                    f"{cls_id},"
                    f"{class_name},"
                    f"{conf:.4f},"  # Format confidence to 4 decimal places
                    f"{int(x_min)},"
                    f"{int(y_min)},"
                    f"{int(x_max)},"
                    f"{int(y_max)}"
                )

    print("--- Processing complete ---")

In [ ]:
if __name__ == "__main__":
    FRAMES_FOLDER = '/Users/yusse/Coding/knight_hacks25/Yolo_frames'

    if FRAMES_FOLDER == '/Users/yusse/Coding/knight_hacks25/Yolo_frames':
        print("="*50)
        print("ERROR: Please update the 'FRAMES_FOLDER' variable in this script")
        print(f"       to point to your directory of video frames.")
        print("="*50)
    else:
        if not os.path.isdir(FRAMES_FOLDER):
            print(f"Error: Folder not found at {FRAMES_FOLDER}")
        else:
            process_frames_folder(FRAMES_FOLDER)

SyntaxError: expected ':' (640311177.py, line 4)